In [38]:
import re
import nltk
from string import punctuation

In [34]:
f = open("CHAVEFolha/folha95/FSP.950101.sgml", "r")
file_content = f.read()

In [458]:
texts = re.findall("(?<=<TEXT>\n)[\w\W]+?(?=<\/TEXT>)", file_content)

# Funções de pré-processamento de texto

In [595]:
def text_lowercase(text):
    return text.lower()

def remove_numbers(text):
    return re.sub(r'\d+', '', text)

def remove_double_periods(text): 
    return re.sub(r'\. *\.', '.', text)

def remove_punctuation(text):
    symbols_pattern = r'[!"#\$%&\'\(\)\*\+,\-\–\/:=\?@\[\\\]\^_`{\|}~º]'
    text = re.sub(symbols_pattern, '', text)
    return text

def remove_multiple_whitespace(text):
    return re.sub(' +', ' ', text)

def remove_stopwords(text):
    stop_words = set(nltk.corpus.stopwords.words('portuguese'))
    tokens = nltk.tokenize.word_tokenize(text)
    clean_text = [token for token in tokens if token not in stop_words]
    return ' '.join(clean_text)

def replace_linebreak_with_period(text):
    return re.sub(' *\n *', ' . ', text)

def insert_tags(text): 
    text = '<s><s>' + text
    text = re.sub(r' *(?<!^)\. *(?!$)| *; *', '</s></s><s><s>', text)
    return re.sub(' *\. *$', '</s></s>', text)

def preprocess_text(text):
    text = text_lowercase(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = replace_linebreak_with_period(text)
    text = remove_multiple_whitespace(text)
    text = remove_double_periods(text)
    text = remove_stopwords(text)
    text = insert_tags(text)
    return text

In [600]:
text = texts[0]
text

'Datafolha revela que 70% esperam governo ótimo ou bom; Collor teve 71% antes de assumir a Presidência \nO sociólogo Fernando Henrique Cardoso, 63, toma possa hoje às 16h30 no Congresso como o 38º presidente do Brasil. Eleito em 3 de outubro com 34,3 milhões de votos, 54,28% dos válidos, FHC sucede a Itamar Augusto Cautiero Franco, 63, de quem foi ministro das Relações Exteriores e da Fazenda.\nO Datafolha revela que 79% apóiam o real, moeda lançada pela equipe de FHC. Para 70%, seu governo será ótimo ou bom. Em 90, 71% tinham a mesma expectativa em relação a Fernando Collor.\nO programa de FHC definiu como prioridades a manutenção do real e as reformas na Constituição. \n'

In [601]:
test = preprocess_text(text)
test

'<s><s>datafolha revela esperam governo ótimo bom</s></s><s><s>collor antes assumir presidência</s></s><s><s>sociólogo fernando henrique cardoso toma possa hoje h congresso presidente brasil</s></s><s><s>eleito outubro milhões votos válidos fhc sucede itamar augusto cautiero franco ministro relações exteriores fazenda</s></s><s><s>datafolha revela apóiam real moeda lançada equipe fhc</s></s><s><s>governo ótimo bom</s></s><s><s>mesma expectativa relação fernando collor</s></s><s><s>programa fhc definiu prioridades manutenção real reformas constituição</s></s>'

In [602]:
print(len(re.findall(r'<s>', test)))
print(len(re.findall(r'</s>', test)))

16
16


In [603]:
re.findall('(?<=<s><s>)[\w\W]+?(?=<\/s><\/s>)', test)

['datafolha revela esperam governo ótimo bom',
 'collor antes assumir presidência',
 'sociólogo fernando henrique cardoso toma possa hoje h congresso presidente brasil',
 'eleito outubro milhões votos válidos fhc sucede itamar augusto cautiero franco ministro relações exteriores fazenda',
 'datafolha revela apóiam real moeda lançada equipe fhc',
 'governo ótimo bom',
 'mesma expectativa relação fernando collor',
 'programa fhc definiu prioridades manutenção real reformas constituição']